In [1]:
import gc
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
from scipy.sparse import csr_matrix
from hicmatrix import HiCMatrix as hm
from scipy.ndimage import median_filter
from scipy.stats import pearsonr

In [2]:
def getMask(raw, minSum=0):
    if (minSum == 0) or (raw is None):
        mask = False
    else:
        raw1 = hm.hiCMatrix(raw[0])
        raw2 = hm.hiCMatrix(raw[1])
        mask = (raw1.matrix + raw2.matrix).todense() < minSum
        del raw1, raw2
    gc.collect()
    return mask

In [3]:
def diagAsBool(a, k):
    """ Return diagonal as a boolean matrix """
    rows, cols = np.diag_indices_from(a)
    if k < 0:
        return rows[-k:], cols[:k]
    elif k > 0:
        return rows[:-k], cols[k:]
    else:
        return rows, cols

In [4]:
def getZmatrix(mat):
    zMatrix = np.zeros(mat.shape)
    nrows = mat.shape[0]
    for diag in range(0, min(500, nrows) + 1):
        indices = diagAsBool(mat, diag)
        values = mat[indices]
        mean = np.nanmean(values)
        std = np.nanstd(values)
        zMatrix[indices] = (values - mean) / std
    zMatrix[np.isnan(zMatrix)] = 0
    zMatrix + zMatrix.T - np.diag(np.diag(zMatrix))
    return zMatrix

In [5]:
minSum = 5
binSize = 20000
chroms = list(range(1, 23)) + ['X']

In [8]:
with open('correlations.tsv', mode='w') as fh:
    print('cell', 'chrom', 'rho', 'p', 'switch', file=fh, sep='\t')
    for chrom in chroms:
        for cell in ['GM12878', 'IMR90', 'H1hESC']:
            if (cell == 'H1hESC') and (chrom == 'X'):
                continue
            raw1 = f'../../{cell}/alleleGRCh37/dat/matrix/chr{chrom}/{binSize}/raw/{cell}_a1-chr{chrom}-{binSize}-SNPsplit-raw.h5'
            raw2 = f'../../{cell}/alleleGRCh37/dat/matrix/chr{chrom}/{binSize}/raw/{cell}_a2-chr{chrom}-{binSize}-SNPsplit-raw.h5'
            mask = getMask([raw1, raw2], minSum=minSum)

            hic1 = hm.hiCMatrix(f'../../{cell}/alleleGRCh37/dat/HiCcompare/chr{chrom}/{binSize}/{cell}_a1-vs-{cell}_a2-adjIF1-obsExp-SNPsplit.h5')
            hic2 = hm.hiCMatrix(f'../../{cell}/alleleGRCh37/dat/HiCcompare/chr{chrom}/{binSize}/{cell}_a1-vs-{cell}_a2-adjIF2-obsExp-SNPsplit.h5')
            nan_bins = set(hic1.nan_bins)
            nan_bins = nan_bins.union(hic2.nan_bins)

            newMatrix = (hic2.matrix - hic1.matrix).todense()
            #z = getZmatrix(newMatrix)
            #del newMatrix
            gc.collect()
            filtered = median_filter(newMatrix, size=3)
            filtered[mask] = np.nan
            #filtered = median_filter(z, size=3)
            # Store 1 / -1 matrix for correlation
            if cell == 'GM12878':
                GMdata = filtered.copy()
            else:
                validPos = ((np.abs(filtered) > 0.2) & (np.abs(GMdata) > 0.2))
                r, p = pearsonr(filtered[validPos], GMdata[validPos])
                switch = (r < 0) and (p < 0.05)
                if switch:
                    filtered *= -1
                print(cell, chrom, r, p, switch, file=fh, sep='\t')
            filtered[mask] = 0
            hic1.setMatrixValues(filtered)
            hic1.maskBins(sorted(nan_bins))
            hic1.save(f'{cell}-chr{chrom}-{binSize}-compare.h5')

            del hic1, hic2, mask, filtered#, z
            gc.collect()

 Overwriting ...

/home/stephen/miniconda3/lib/python3.9/site-packages/tables/atom.py:362: FlavorWarning: support for unicode type is very limited, and only works for strings that can be cast as ascii
  warnings.warn("support for unicode type is very limited, and "
 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting ...

 Overwriting .

In [11]:
# Perform subtraction with LOESS to loo at 4q32.2 locus
chrom = 4
for cell in ['GM12878', 'IMR90', 'H1hESC']:
    raw1 = f'../../{cell}/alleleGRCh37/dat/matrix/chr{chrom}/{binSize}/raw/{cell}_a1-chr{chrom}-{binSize}-SNPsplit-raw.h5'
    raw2 = f'../../{cell}/alleleGRCh37/dat/matrix/chr{chrom}/{binSize}/raw/{cell}_a2-chr{chrom}-{binSize}-SNPsplit-raw.h5'
    mask = getMask([raw1, raw2], minSum=5)

    hic1 = hm.hiCMatrix(raw1)
    hic2 = hm.hiCMatrix(raw2)
    nan_bins = set(hic1.nan_bins)
    nan_bins = nan_bins.union(hic2.nan_bins)

    newMatrix = (hic2.matrix - hic1.matrix).todense().astype(float)
    newMatrix[mask] = np.nan 
    z = getZmatrix(newMatrix)
    del newMatrix
    gc.collect()
    filtered = median_filter(z, size=3)
    # Store 1 / -1 matrix for correlation
    if cell == 'GM12878':
        GMdata = filtered.copy()
    else:
        validPos = ((np.abs(filtered) > 1) & (np.abs(GMdata) > 1))
        r, p = pearsonr(filtered[validPos], GMdata[validPos])
        switch = (r < 0) and (p < 0.05)
        if switch:
            filtered *= -1

    hic1.setMatrixValues(filtered)
    hic1.maskBins(sorted(nan_bins))
    hic1.save(f'{cell}-chr{chrom}-{binSize}-compare-noLoess.h5')

    del hic1, hic2, mask, filtered#, z
    gc.collect()

INFO:hicmatrix.HiCMatrix:Number of poor regions to remove: 186 {'chr4': 186}
INFO:hicmatrix.HiCMatrix:found existing 181 nan bins that will be included for masking 
INFO:hicmatrix.HiCMatrix:masked bins were restored

 Overwriting ...

/home/stephen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys
/home/stephen/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/stephen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
INFO:hicmatrix.HiCMatrix:Number of poor regions to remove: 202 {'chr4': 202}
INFO:hicmatrix.HiCMatrix:found existing 195 nan bins that will be included for masking 
INFO:hicmatrix.HiCMatrix:masked bins were restored

INFO:hicmatrix.HiCMatrix:Number of poor regions to remove: 185 {'chr4': 185}
INFO:hicmatrix.HiCMatrix:f

In [24]:
validPos = ((np.abs(filtered) > 1.5) & (np.abs(GMdata) > 1.5))
r, p = pearsonr(filtered[validPos] / np.abs(filtered[validPos]), GMdata[validPos] / np.abs(GMdata[validPos]) )
print(r, p)

-0.06666666666666671 0.8753698765432096


In [31]:
pearsonr(np.diag(filtered, 3), np.diag(GMdata, 3))

(0.009584416933134101, 0.6276792545302561)